In [2]:
import sys
import tweepy
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
from wordcloud import WordCloud, STOPWORDS
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from credentials import *

In [6]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth)

In [8]:
tweets = api.user_timeline('@RamireJRY', count=5, tweet_mode='extended')
for t in tweets:
    print(t.full_text)
    print()

@udemy_es @Fernando_Her85

